<a href="https://colab.research.google.com/github/UmerAzmi/Colorizer/blob/main/Colorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Colorizer***
```
Before running change runtime type to Python
And Hardware Accelerator to GPU
```



In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
cd DeOldify

In [ ]:
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
!pip install -r requirements-colab.txt

In [ ]:
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

In [ ]:
import torch

old_torch_load = torch.load
def torch_load_patch(*args, **kwargs):
    kwargs['weights_only'] = False
    return old_torch_load(*args, **kwargs)
torch.load = torch_load_patch

In [ ]:
colorizer = get_image_colorizer(artistic=True)

In [ ]:
!apt-get install -y poppler-utils
!pip install pdf2image

In [ ]:
from pdf2image import convert_from_path

pdf_path = "/content/a.pdf"  # Path of the uploaded image or PDF (Upload the files in /content)
pages = convert_from_path(pdf_path, dpi=200)

for i, page in enumerate(pages):
    page.save(f"/content/PDF2PNG/page_{i+1}.png", "PNG")

print("PDF converted into images")

In [ ]:
import glob, os

input_files = sorted(glob.glob("/content/PDF2PNG/page_*.png")) #Create a Folder named PDF2PNG

for file in input_files:
    print("Colorizing:", file)
    base = os.path.basename(file)

    colorizer.plot_transformed_image(
        file,
        render_factor=28,
        figsize=(20,20)
    )

print("All pages colorized")

In [ ]:
from PIL import Image
import glob
import os

colored_files = glob.glob("/content/DeOldify/result_images/page_*.png")

def sort_key(filename):
    return int(os.path.basename(filename).split('_')[1].split('.')[0])

colored_files_sorted = sorted(colored_files, key=sort_key)

print("Found pages:", colored_files_sorted)

images = [Image.open(f).convert("RGB") for f in colored_files_sorted]

if images:
    images[0].save("/content/OutputPDF.pdf", save_all=True, append_images=images[1:])
    print("Colored PDF created: /content/OutputPDF.pdf")
else:
    print("No images found to create PDF.")

In [ ]:
import shutil

shutil.rmtree('/content/DeOldify/result_images/page_*.png', ignore_errors=True)
shutil.rmtree('/content/colored_pages', ignore_errors=True)
for file in glob.glob("/content/PDF2PNG/page_*.png"):
    os.remove(file)

print("All temporary images deleted!")

In [ ]:
!rm -rf DeOldify
print("DeOldify folder deleted!")

In [ ]:
!pip install pypdf

In [ ]:
from pypdf import PdfWriter

merger = PdfWriter()
pdf_files = ["/content/1.pdf", "/content/2.pdf", "/content/3.pdf"]

for pdf in pdf_files:
    merger.append(pdf)

output_filename = "/content/combined_output.pdf"
merger.write(output_filename)
merger.close()

print(f"PDFs combined into {output_filename}")